In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_openml
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.tree import DecisionTreeClassifier

data = fetch_openml('mnist_784', version=1)#Get data from https://www.openml.org/d/554
dfData = pd.DataFrame(np.c_[data["data"],data["target"]],columns = data["feature_names"]+["target"])

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [2]:
stratSplit = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)

for train_index, test_index in stratSplit.split(dfData[data["feature_names"]], dfData["target"]):
    X_train = dfData[data["feature_names"]].iloc[train_index]
    X_test = dfData[data["feature_names"]].iloc[test_index]

    y_train = dfData["target"].iloc[train_index]
    y_test = dfData["target"].iloc[test_index]

In [3]:
from sklearn.ensemble import BaggingClassifier

#soft voting by default if predict_proba method exists
bag_clf = BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=3,
                                                      max_features='sqrt',
                                                      splitter='random'),
                            n_estimators=30,  #30 different decision tree classifiers
                            max_samples=0.8,  # int(e.g.100 = 100 different samples)/float, default with replacement, size of the subset of the training data we give to each model during training
                                              # maximum training set sample size compared to original training set (80% of original training set)
                            max_features=0.9, # int(e.g. 100 = 100 different features)/float, default without replacement, define the random subspaces
                                              # maximum feature size compared to original number of features
                            bootstrap=True,   # bagging (= set to True)/pasting (= set to False) (affects max_samples behaviour)
                            bootstrap_features=False, # if feature selection should use bagging (max_features), random sampling the features without replacement
                            oob_score=False,  # perform oob scoring (oob=out-of-bag), it will slightly increase training times just because then youalso have to perform the predictions and evaluate them
                            warm_start=False,
                            n_jobs=2,
                            random_state=0,
                            verbose=0
                            )

bag_clf.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=3,
                                                        max_features='sqrt',
                                                        splitter='random'),
                  max_features=0.9, max_samples=0.8, n_estimators=30, n_jobs=2,
                  random_state=0)

In [4]:
print(len(bag_clf.estimators_)) # array of trained estimators
print(len(bag_clf.estimators_samples_)) # array of sample subsets used to train each estimator
print(len(bag_clf.estimators_features_)) # array of feature subsets used to train each estimator

# #when oob_score is true we also get the out-of-bag score
# bag_clf.oob_score_
# bag_clf.oob_decision_function_

30
30
30


In [6]:
#iteration over all of our estimators because each estimator is trained on a subset of features
#we need to make sure we iterate over the features
scores = []
for est,features in zip(bag_clf.estimators_,bag_clf.estimators_features_):
    scores.append(est.score(X_test.values[:, features], y_test.astype(np.float64))) #score it on the testing set only using the relevant features
    # Sklearn transform str for of integers to np.float64 internally (because it's treated originally as a float)

In [7]:
print('Avg. estimator performance:',np.mean(scores)) #=accuracy
print('Estimaor performance std. dev.:',np.std(scores))

Avg. estimator performance: 0.35524999999999995
Estimaor performance std. dev.: 0.025101865597253623


In [8]:
bag_clf.score(X_test, y_test)

0.6851428571428572

Even though what we're doing here is we're doing a ton of randomization and eachdecision tree only gets a subset of the samples and a subset of the features.And it even performs splitting randomly ratherthan trying to find the best splitter,we still get a huge performance boost.UmAnd yeah, that's the power that we can get from ensemble methods. Don't forget to use grid search to find optimal parameters (finding a good balance between training time and performance) which can lead to a better final performance (here: better than 0.685).